Importing the dependencies

In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# from joblib import Parallel, delayed
# from concurrent.futures import ThreadPoolExecutor

Data Collection and Pre-Processing

In [2]:
# loading the data from the csv file to a pandas dataframe


movies=pd.read_csv('movies.csv')





C:\Users\East-Sound\AppData\Local\Temp\ipykernel_10376\920090155.py:4: DtypeWarning: Columns (1,4,9,13,14,19,20,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,

In [3]:
# simulate user ratings
np.random.seed(42)
movies['user_rating'] = np.random.randint(1, 6, size=len(movies))

In [4]:
# printing the first 5 rows of the dataframe
movies.head()


index     budget                                    genres   
0      0  237000000  Action Adventure Fantasy Science Fiction  \
1      1  300000000                  Adventure Fantasy Action   
2      2  245000000                    Action Adventure Crime   
3      3  250000000               Action Crime Drama Thriller   
4      4  260000000          Action Adventure Science Fiction   

                                       homepage      id   
0                   http://www.avatarmovie.com/   19995  \
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language   
0  culture clash future space war space colony so...                en  \
1  ocean drug abuse exotic island east india trad...                en   
2         spy based on novel secret agent sequel mi6                en   
3  dc comics crime fighter terrorist secret ident...                en   
4  based on novel mars medallion space travel pri...                en   

                             original_title   
0                                    Avatar  \
1  Pirates of the Caribbean: At World's End   
2                                   Spectre   
3                     The Dark Knight Rises   
4                               John Carter   

                                            overview  popularity  ...   
0  In the 22nd century, a paraplegic Marine is di...  150.437577  ...  \
1  Captain Barbossa, long believed to be dead, ha...  139.082615  ...   
2  A cryptic message from Bond’s past sends him o...  107.376788  ...   
3  Following the death of District Attorney Harve...   112.31295  ...   
4  John Carter is a war-weary, former military ca...   43.926995  ...   

  Unnamed: 1256 Unnamed: 1257 Unnamed: 1258 Unnamed: 1259 Unnamed: 1260   
0           NaN           NaN           NaN           NaN           NaN  \
1           NaN           NaN           NaN           NaN           NaN   
2           NaN           NaN           NaN           NaN           NaN   
3           NaN           NaN           NaN           NaN           NaN   
4           NaN           NaN           NaN           NaN           NaN   

  Unnamed: 1261 Unnamed: 1262 Unnamed: 1263 Unnamed: 1264 user_rating  
0           NaN           NaN           NaN           NaN           4  
1           NaN           NaN           NaN           NaN           5  
2           NaN           NaN           NaN           NaN           3  
3           NaN           NaN           NaN           NaN           5  
4           NaN           NaN           NaN           NaN           5  

[5 rows x 1266 columns]

In [5]:
# number of rows and columns in the data frame

movies.shape

(4809, 1266)

In [6]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(movies, test_size=0.2, random_state=42)

In [7]:
# selecting the relevant features for recommendation

selected_features = ['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [8]:
# replacing the null values with null strings

for feature in selected_features:
  movies[feature] = movies[feature].fillna('')

In [9]:
# combining all the 5 selected features

combined_features = movies['genres']+' '+ movies['keywords']+' '+ movies['tagline']+' '+ movies['cast']+' '+ movies['director']
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4804    Action Crime Thriller united states\u2013mexic...
4805    Comedy Romance  A newlywed couple's honeymoon ...
4806    Comedy Drama Romance TV Movie date love at fir...
4807      A New Yorker in Shanghai Daniel Henney Eliza...
4808    Documentary obsession camcorder crush dream gi...
Length: 4809, dtype: object


In [10]:
# Importing necessary libraries
import numpy as np
from joblib import Parallel, delayed
from scipy import sparse

# Converting text data to a matrix of features (TF-IDF vectors)
# converting the text data (combined_features) to a matrix of features (feature vectors)
vectorizer = TfidfVectorizer()

# Function to calculate TF-IDF vectors for a subset of data
def calculate_tfidf_subset(data_subset, vectorizer):
    combined_features_subset = data_subset['genres'] + ' ' + data_subset['keywords'] + ' ' + data_subset['tagline'] + ' ' + data_subset['cast'] + ' ' + data_subset['director']
    return vectorizer.transform(combined_features_subset)

# Splitting the data into chunks for parallel processing
num_cores = 4  # You can adjust the number of cores based on your machine
chunks = np.array_split(movies, num_cores)

# Fitting the vectorizer on the entire dataset
vectorizer.fit(combined_features)

# Parallel processing of TF-IDF vectorization
results = Parallel(n_jobs=num_cores)(delayed(calculate_tfidf_subset)(chunk, vectorizer) for chunk in chunks)

# Combining the results
feature_vectors = sparse.vstack(results)
print(feature_vectors)

  (0, 17299)	0.2019755488126334
  (0, 17016)	0.23642216288179602
  (0, 17007)	0.1282251636064218
  (0, 16677)	0.19842983735418923
  (0, 16596)	0.12550744845505796
  (0, 15270)	0.0709833692900418
  (0, 14617)	0.15151416868595385
  (0, 14387)	0.3396344162602317
  (0, 14280)	0.21391560760772346
  (0, 14073)	0.205956457941733
  (0, 13608)	0.10365929555963475
  (0, 13358)	0.15022036822291404
  (0, 13328)	0.2177400340451317
  (0, 13033)	0.19423432006987434
  (0, 11511)	0.27209420901160897
  (0, 11200)	0.09051396620449952
  (0, 10237)	0.16059231591328046
  (0, 8764)	0.22708109846346383
  (0, 7763)	0.11281946761216571
  (0, 5843)	0.16467965954169428
  (0, 5444)	0.10365929555963475
  (0, 5281)	0.11110189871032784
  (0, 4952)	0.24024658931920426
  (0, 3685)	0.21391560760772346
  (0, 3231)	0.24958765373753639
  :	:
  (4807, 10937)	0.13509136428761018
  (4807, 7482)	0.11312822232180882
  (4807, 7276)	0.30263451355310916
  (4807, 6942)	0.28866712901660546
  (4807, 4842)	0.2471929040013495
  (4807, 

Cosine Similarity

In [11]:
# getting the similarity scores using "cosine_similarity"

similarity = cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.0722257  0.03774545 ... 0.         0.         0.        ]
 [0.0722257  1.         0.03282986 ... 0.03576038 0.         0.        ]
 [0.03774545 0.03282986 1.         ... 0.         0.05370221 0.        ]
 ...
 [0.         0.03576038 0.         ... 1.         0.         0.02651822]
 [0.         0.         0.05370221 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651822 0.         1.        ]]


In [12]:
# number of rows and columns in the similarity

print(similarity.shape)


(4809, 4809)


In [13]:
accuracy = 0
total_users = 0

for user_id, user_data in test_data.iterrows():
    movie_name = user_data['title']
    list_of_all_movies = movies['title'].fillna('').tolist()
    find_close_match = difflib.get_close_matches(movie_name,list_of_all_movies)

    if find_close_match:
        close_match = find_close_match[0]
        index_of_the_movie =movies[movies.title == close_match]['index'].values[0]
        
        
        similarity_score = list(enumerate(similarity[int(index_of_the_movie)]))
        sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)

        # Assuming the actual user rating is in the 'user_rating' column
        actual_rating = user_data['user_rating']

        # Find the recommendation rank of the actual movie in the sorted list
        rank = next((i + 1 for i, movie in enumerate(sorted_similar_movies) if movies[movies.index == movie[0]]['title'].values[0] == movie_name), None)

        if rank is not None:
            accuracy += 1 / rank  # Higher rank contributes less to accuracy
            total_users += 1
            

# Calculate the percentage of test and training data
percentage_test_data = len(test_data) / len(movies) * 100
percentage_train_data = len(train_data) / len(movies) * 100

print(f"Percentage of Test Data: {percentage_test_data}%")
print(f"Percentage of Training Data: {percentage_train_data}%")

# Calculate the average accuracy
average_accuracy = accuracy / total_users if total_users > 0 else 0
print(f"Average Accuracy: {average_accuracy}")


Percentage of Test Data: 20.004158868787687%
Percentage of Training Data: 79.9958411312123%
Average Accuracy: 0.996882203515891


Getting the movie name from the user

In [14]:
# getting the movie name from the user

movie_name = input(' Enter your favourite movie name : ')

In [15]:
# creating a list of all movies given in the dataset

list_of_all_movies = movies['title'].fillna('').tolist()
print(list_of_all_movies)


['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', " 'department': 'Sound'", 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Lega

In [16]:
# finding the close match for the (movie_name) given by the user

find_close_match = difflib.get_close_matches(movie_name, list_of_all_movies)
print(find_close_match)

['Batman', 'Batman', 'Catwoman']


In [17]:
close_match = find_close_match[0]
print(close_match)

Batman


In [18]:
# finding the index of the movie with title = (close_match)

index_of_the_movie = movies[movies.title == close_match]['index'].values[0]
print(index_of_the_movie)

1365


In [19]:
# getting a list of (similarity_score) of all the movies

similarity_score = list(enumerate(similarity[int(index_of_the_movie)]))
print(similarity_score)

[(0, 0.025328697160637695), (1, 0.04985869088565574), (2, 0.013607589377592374), (3, 0.20439114029752078), (4, 0.024940000189236176), (5, 0.11534798628768905), (6, 0.0), (7, 0.005525335952541738), (8, 0.030925574287098453), (9, 0.08976889550931216), (10, 0.11242320399288504), (11, 0.011814554377985778), (12, 0.038268851181621925), (13, 0.013273248486436047), (14, 0.12815141815653358), (15, 0.010436320263248509), (16, 0.005239268856189682), (17, 0.016023815864817898), (18, 0.041537227916086904), (19, 0.02970355088278996), (20, 0.025223941197325003), (21, 0.004594219580564547), (22, 0.017931879572013323), (23, 0.02488631105635774), (24, 0.04187989249882731), (25, 0.0), (26, 0.005702647356271324), (27, 0.013116104395171052), (28, 0.015213211318986746), (29, 0.0), (30, 0.005727769428806948), (31, 0.113374802508678), (32, 0.022919473935899413), (33, 0.08995590414503288), (34, 0.005491094331329686), (35, 0.0), (36, 0.025780937639920558), (37, 0.043227393907748095), (38, 0.022080744343284428)

In [20]:
len(similarity_score)

4809

In [21]:
# sorting the (similarity_score) of all the movies => "Descending"

sorted_similar_movies = sorted(similarity_score, key= lambda x:x[1], reverse= True)
print(sorted_similar_movies)

[(1365, 1.0), (433, 0.43116678663720176), (211, 0.25739652365225696), (3, 0.20439114029752078), (120, 0.19262989262819802), (66, 0.17755792086734112), (1518, 0.14706644956178272), (818, 0.1441618128745309), (2536, 0.1373752526423472), (1022, 0.13715047270227973), (478, 0.1321854988767841), (758, 0.13216330626736136), (281, 0.12996549136285632), (14, 0.12815141815653358), (73, 0.12267832847107706), (2319, 0.11835419850883203), (2387, 0.11753699319720314), (5, 0.11534798628768905), (2864, 0.11413805017776628), (4189, 0.11357884468282277), (31, 0.113374802508678), (2661, 0.11272408210144207), (10, 0.11242320399288504), (1040, 0.11196875012429089), (2691, 0.10981521231451943), (875, 0.1072380796090968), (42, 0.10650712467051823), (1302, 0.1058575625378233), (443, 0.10277737654674986), (1483, 0.10200903017976529), (303, 0.10155726686906205), (1480, 0.10080557859779404), (1007, 0.10054235897593451), (1090, 0.10023313734042023), (2759, 0.09952101235156519), (307, 0.09885179495220406), (2811, 

In [22]:
# print the name of similar movies based on the index

print('Movies suggested for you : \n')

i = 1
for movie in sorted_similar_movies:
  
  index = movie[0]
  title_from_index = movies[movies.index==index]['title'].values[0]
  if (i<=10):
    
    print(i, '-', title_from_index)
    i+=1

Movies suggested for you : 

1 - Batman
2 - Batman Returns
3 - Batman & Robin
4 - The Dark Knight Rises
5 - Batman Begins
6 - The Dark Knight
7 - A History of Violence
8 - Superman
9 - Beetlejuice
10 - Bedazzled
